<a href="https://colab.research.google.com/github/alexandergribenchenko/DS_LATAM_Test/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Cuál es la proabilidad de que se retrase mi vuelo?
**Data Scientist Challenge - LATAM Airlines (Presentado por: Alexander Ortega, Marzo 2023)**



Este notebook presenta la solución al reto analítico propuesto por LATAM que consiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago de Chile (SCL).
 
El notebook está compuesto de 5 secciones que dan respuesta a cada uno de los ítems descritos en la sección de `Desafío`:
 
- **Sección 01. Distribución de los datos:** se analiza la distribución de los datos para cada uno de los features que componen el dataset.
- **Sección 02. Generación de Columnas Adicionales:** se genera un dataset denominado `synthetic_features.csv` con las columnas adicionales que dan cuenta de la temporada (alta o baja), el retraso en minutos, la etiqueta de atrasado (si, no) si el retraso es superior a 15 minutos y por último en periodo del día (mañana, tarde o noche).
- **Sección 03. Comportamiento del atraso vs features relevantes:** se describe la relación entre la variable de retraso y las variables destino, aerolínea, mes del año, día de la semana, temporada y tipo de vuelo. A partir de sus resultados se discuten cuáles son las que pueden presentar mayor relevancia en el modelo.
- **Sección 04. Entrenamiento de modelos propuestos:** se presenta el pipeline que compone el procesamiento de datos y entrenamiento de 3 modelos base: Logistic Regression, Decision Tree Classifier y xgboost.
- **Sección 05. Evaluación de modelos propuestos:** se discute la métrica elegida para evaluar el performance del modelo y se presentan los resultados para cada uno de los modelos. Así mismo se discuten las variables que mayor influencia presentan en la predicción y algunas alternativas para mejorar su performance.


# A. Líbrerías

In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

# B. Dataframe raw

In [2]:
path_github = 'https://raw.githubusercontent.com/alexandergribenchenko/DS_LATAM_Test/main/data/dataset_SCL.csv'

In [3]:
df_raw = pd.read_csv(path_github, dtype=object)
df_raw

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  object
 11  MES        68206 non-null  object
 12  AÑO        68206 non-null  object
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: object(18)
memory usage: 9.4+ MB


# Sección 01. Distribución de los datos

# Sección 02. Generación de Columnas Adicionales

En esta sección se genera un dataset denominado `synthetic_features.csv` con las columnas adicionales que dan cuenta de la temporada (alta o baja), el retraso en minutos, la etiqueta de atrasado (si o no) si el retraso es superior a 15 minutos y por último en periodo del día (mañana, tarde o noche).


### ---> Aproximación de la solución:
Esta sección se desarrolla empleando en uso transformadores y pipelines bajo la aproximación de programación OOP. Si bien el procedimiento para un proceso de transformación simple puede prescindir de ello, la idea de esta aproximación presenta algunas ventajas:
- **Estandarización del desarrollo:**  se pueda contar con transformadores y pipelines que estandarizan los procedimientos garantizando que se estarán haciendo de la misma manera sobre cualquier conjunto de datos. 
- **Facilita pruebas unitarias**: dado que los transformadores y pipelines cumplen con tareas específicas se pueden facilitar las pruebas unitarias sobre el desarrollo.
- **Parametrización y agilidad en el desarrollo:** permite que el desarrollo sea fácilmente parametrizable pues cambiando únicamente las entradas de los transformadores se pueden crear nuevos pipelines para necesidades específicas.

## 02.01. Definición de los Transformadores

### ---> FeatureSelector
Filtra las columnas seleccionadas en el dataset de entrada.

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_FeatureSelector):
        self.feature_names = params_FeatureSelector['feature_names'] 
    
    def fit(self, X, y = None):
        return self 
    
    def transform(self, X, y = None):
      X_output = X[self.feature_names]
      return X_output

### ---> DateType
Convierte a formato `datetime` las columnas seleccionadas.

In [6]:
class DateType(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_FeatureSelector):
        self.cols_dates = params_DateType['cols_dates'] 
    
    def fit(self, X, y = None):
        return self 
    
    def transform(self, X, y = None):
      X_output = X.copy()
      for col in self.cols_dates:
        X_output[col] = pd.to_datetime(X_output[col])
      return X_output

### ---> TemporadaAlta
Crea la columna `temporada_alta` a partir de una columna de fecha que se le especifíque. 

EL criterio usado es: 
- temporada_alta(1): si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep.
- temporada_alta(0): si es diferente a las fechas descritas en 1.

In [7]:
class TemporadaAlta(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_TemporadaAlta):
        self.columna_fecha = params_TemporadaAlta['columna_fecha'] 
    
    def fit(self, X, y = None):
        return self

    def fun_temporada_alta(self, fecha):
      mes = fecha.month
      dia = fecha.day
      if (mes == 12 and dia >= 15) or \
      (mes in [1, 2]) or \
      (mes == 3 and dia <= 3) or \
      (mes == 7 and (dia >= 15 and dia <= 31)) or \
      (mes == 9 and (dia >= 11 and dia <= 30)):
          return 1
      else:
          return 0
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['temporada_alta'] = X_output[self.columna_fecha].apply(self.fun_temporada_alta)
      return X_output

### ---> DifMin
Crea la columna `dif_min` que es la diferencia en minutos entre Fecha-O y Fecha-I. Básicamante el tiempo en minutos que se retraso el vuelo.

In [8]:
class DifMin(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_DifMin):
        self.empty = params_DifMin 
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['dif_min'] = (X_output['Fecha-O']-X_output['Fecha-I'])/pd.Timedelta(minutes=1)
      return X_output

### ---> Atraso15
Crea la columna `atraso_15` a partir de una columna `dif_min`. Esta columna es la que servirá como target en nuestro modelo de clasificación.

EL criterio usado es: 
- temporada_alta(1): si dif_min > 15
- temporada_alta(0): si dif_min <= 15

In [9]:
class Atraso15(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_Atraso15):
        self.empty = params_Atraso15 
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['atraso_15'] = (X_output['dif_min'] > 15).astype(int)
      return X_output

### ---> PeriodoDia
Crea la columna `periodo_dia` a partir de una columna de fecha que se le especifíque. 

EL criterio usado es: 
- periodo_dia(mañana): entre 5:00 y 11:59 hrs.
- periodo_dia(tarde): entre 12:00 y 18:59 hrs.
- periodo_dia(noche): entre 19:00 y 4:59 hrs.



In [10]:
class PeriodoDia(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_PeriodoDia):
        self.columna_fecha = params_PeriodoDia['columna_fecha'] 
    
    def fit(self, X, y = None):
        return self

    def fun_periodo_dia(self, fecha):
      hora = fecha.hour
      if hora >= 5 and hora <= 11:
          return 'mañana'
      elif hora >= 12 and hora <= 18:
          return 'tarde'
      else:
          return 'noche'
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['periodo_dia'] = X_output[self.columna_fecha].apply(self.fun_periodo_dia)
      return X_output

### ---> OrderOutput
Filtra las columnas seleccionadas en el dataset de entrada. Lo usamos para obtener las columnas que deseamos en el dataset de salida.

In [11]:
class OrderOutput(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_OrderOutput):
        self.feature_names = params_OrderOutput['feature_names'] 
    
    def fit(self, X, y = None):
        return self 
    
    def transform(self, X, y = None):
      X_output = X[self.feature_names]
      return X_output

## 02.02. Definición de Pipeline

### ---> Parámetros de los transformadores
Definimos cada uno de los parámetros de entrada que deben tener los transformadores para nuestro pipeline.

In [12]:
params_FeatureSelector = {}
params_FeatureSelector['feature_names']= ['Fecha-I', 
                                          'Des-I',
                                          'Emp-I', 
                                          'TIPOVUELO', 
                                          'Fecha-O',
                                         ]

params_DateType = {}
params_DateType['cols_dates']= ['Fecha-I', 'Fecha-O']

params_TemporadaAlta = {}
params_TemporadaAlta['columna_fecha']= 'Fecha-I'

params_DifMin = {}

params_Atraso15 = params_DifMin

params_PeriodoDia = params_TemporadaAlta

params_OrderOutput = {}
params_OrderOutput['feature_names']= ['temporada_alta',
                                      'dif_min',
                                      'atraso_15',	
                                      'periodo_dia'
                                         ]

### ---> Instanciamiento de los Transformadores

Instanciamos nuestros transformadores.

In [13]:
Transformer_FeatureSelector = FeatureSelector(params_FeatureSelector)
Transformer_DateType = DateType(params_DateType)
Transformer_TemporadaAlta = TemporadaAlta(params_TemporadaAlta)
Transformer_DifMin = DifMin(params_DifMin)
Transformer_Atraso15 = Atraso15(params_Atraso15)
Transformer_PeriodoDia = PeriodoDia(params_PeriodoDia)
Transformer_OrderOutput = OrderOutput(params_OrderOutput)

### ---> Instanciamiento del Pipeline
Instanciamos el pipeline que dará origen al dataset `synthetic_features.csv`.

In [14]:
pipeline_synthetic_features = Pipeline(steps=[('NameFeatureSelector', Transformer_FeatureSelector),
                              ('NameDateType', Transformer_DateType),
                              ('NameTemporadaAlta', Transformer_TemporadaAlta),
                              ('NameDifMin', Transformer_DifMin),
                              ('NameDateAtraso15', Transformer_Atraso15),
                              ('NamePeriodoDia', Transformer_PeriodoDia),
                              ('NameOrderOutput', Transformer_OrderOutput)
                              ])

## 02.03. Generación del dataframe solicitado
Generamos el dataset solicitado con las 4 columnas y lo almacenamos. Esta dataset posteriormente se mueve y queda almacenado en el folder `data` del repositorio principal.

**Nota**: se asume que se desean únicamente las 4 columnas descritas en este archivo. Si se desearan otras adicionales solo se deberían cambiar los parámetros de los transformadores FeatureSelector y OrderOutput.

In [15]:
df_synthetic_features = pipeline_synthetic_features.transform(df_raw)
df_synthetic_features

,temporada_alta,dif_min,atraso_15,periodo_dia
0,1,3.0,0,noche
1,1,9.0,0,noche
2,1,9.0,0,noche
3,1,3.0,0,noche
4,1,-2.0,0,noche
...,...,...,...,...
68201,1,46.0,1,tarde
68202,1,16.0,1,tarde
68203,1,40.0,1,tarde
68204,1,13.0,0,tarde


In [17]:
df_synthetic_features.to_csv('synthetic_features.csv', index=False)

In [16]:
fin_seccion_02

NameError: ignored

# Sección 03. Composición de la tasa de atraso

In [ ]:
df_atraso = df[['SIGLADES','OPERA', 'MES', 'DIANOM', 'temporada_alta', 'TIPOVUELO', 'atraso_15']]
df_atraso['temporada_alta'] = df_atraso['temporada_alta'].astype(str)
df_atraso.head(3)

In [ ]:
def dataframe_atraso_por(df_atraso, col_interes):
  df_output = df_atraso.groupby([col_interes], as_index=False).\
                            agg(porcentaje_vuelos_con_atraso = ('atraso_15', lambda x: round(100*(x == 1).sum()/x.count(),2)),
                                porcentaje_vuelos_sin_atraso = ('atraso_15', lambda x: round(100*(x == 0).sum()/x.count(),2)),
                                vuelos_con_atraso = ('atraso_15', lambda x: (x == 1).sum()), 
                                vuelos_sin_atraso = ('atraso_15', lambda x: (x == 0).sum()),
                                vuelos_total=('atraso_15', 'count'),
                                ).\
                            sort_values(by=[col_interes], ascending=[True])

  return df_output

In [ ]:
def plot_absoluto_porcentual(df_atraso_por_destino, col_interes, width, length):

  order_absoluto = list(df_atraso_por_destino.sort_values(by=['vuelos_con_atraso'], ascending=[False])[col_interes])
  order_porcentual = list(df_atraso_por_destino.sort_values(by=['porcentaje_vuelos_con_atraso'], ascending=[False])[col_interes])

  fig, axs = plt.subplots(nrows=1, ncols=2, sharey=False)
  fig.set_size_inches(width, length)
  fig.subplots_adjust(wspace=0.5)

  # Plot the total crashes
  sns.set_color_codes("pastel")
  sns.barplot(ax=axs[0], x="vuelos_total", y=col_interes, data=df_atraso_por_destino,
              label="Total", color="g", order=order_absoluto)

  # Plot the crashes where alcohol was involved
  sns.set_color_codes("muted")
  sns.barplot(ax=axs[0], x="vuelos_sin_atraso", y=col_interes, data=df_atraso_por_destino,
              label="vuelos_sin_atraso", color="b", order=order_absoluto )

  # Plot the crashes where alcohol was involved
  # sns.set_color_codes('pastel')
  sns.barplot(ax=axs[0], x="vuelos_con_atraso", y=col_interes, data=df_atraso_por_destino,
              label="vuelos_con_atraso", color="r", order=order_absoluto)

  # Add a legend and informative axis label
  axs[0].legend(ncol=3, loc="upper right", bbox_to_anchor=(1,1.05), frameon=True)
  axs[0].set_xlabel("Cantidad de vuelos")
  axs[0].set_ylabel("")

  # Grafico derecho: Porcentual

  # Plot the crashes where alcohol was involved
  sns.set_color_codes("muted")
  sns.barplot(ax=axs[1], x="porcentaje_vuelos_sin_atraso", y=col_interes, data=df_atraso_por_destino,
              label="porcentaje_vuelos_sin_atraso", color="b", order=order_porcentual, alpha=0.7)

  # Plot the crashes where alcohol was involved
  # sns.set_color_codes('pastel')
  sns.barplot(ax=axs[1], x="porcentaje_vuelos_con_atraso", y=col_interes, data=df_atraso_por_destino,
              label="porcentaje_vuelos_con_atraso", color="r", order=order_porcentual, alpha=0.7)

  axs[1].legend(ncol=2, loc="upper right", bbox_to_anchor=(1,1.05), frameon=True)
  axs[1].set_xlabel("Porcentaje")
  axs[1].set_ylabel("")
  axs[1].set_xlim(0, 100)

  plt.show()

## 03.01. Tasa de atraso por destino

In [ ]:
df_atraso_por_destino = dataframe_atraso_por(df_atraso, 'SIGLADES')
df_atraso_por_destino

In [ ]:
plot_absoluto_porcentual(df_atraso_por_destino, 'SIGLADES', 15, 15)

## 03.02. Tasa de atraso por aerolinea

In [ ]:
df_atraso_por_aerolinea = dataframe_atraso_por(df_atraso, 'OPERA')
df_atraso_por_aerolinea

In [ ]:
plot_absoluto_porcentual(df_atraso_por_aerolinea, "OPERA", 15, 10)

## 03.03. Tasa de atraso por mes del año

In [ ]:
df_atraso_por_mes = dataframe_atraso_por(df_atraso, 'MES')
df_atraso_por_mes

In [ ]:
plot_absoluto_porcentual(df_atraso_por_mes, 'MES', 15, 8)

## 03.04. Tasa de atraso por día de la semana

In [ ]:
df_atraso_por_dia_semana = dataframe_atraso_por(df_atraso, 'DIANOM')
df_atraso_por_dia_semana

In [ ]:
plot_absoluto_porcentual(df_atraso_por_dia_semana, 'DIANOM', 15, 8)

## 03.05. Tasa de atraso por temporada

In [ ]:
df_atraso_por_temporada = dataframe_atraso_por(df_atraso, 'temporada_alta')
df_atraso_por_temporada

In [ ]:
plot_absoluto_porcentual(df_atraso_por_temporada, 'temporada_alta', 15, 4)

## 03.06. Tasa de atraso por tipo de vuelo

In [ ]:
df_atraso_por_tipo_vuelo = dataframe_atraso_por(df_atraso, 'TIPOVUELO')
df_atraso_por_tipo_vuelo

In [ ]:
plot_absoluto_porcentual(df_atraso_por_tipo_vuelo, 'TIPOVUELO', 15, 4)

## 03.07. Expectativa en relación a las variables más influyentes en el modelo

In [ ]:
df_raw

In [ ]:
df_raw['Vlo-I'].nunique()

In [ ]:
df_raw['Des-I'].nunique()

In [ ]:
df_raw.nunique().reset

In [ ]:
df_conteo = df_raw.groupby(['Des-I'], as_index=False).\
           agg(unique_col=('Vlo-I', lambda x: x.nunique()),
               conca=('Vlo-I', lambda x: ','.join(x.unique()))).\
           sort_values(by=['unique_col'], ascending=[False])
df_conteo

In [ ]:
df.columns

In [ ]:
columnas_modelo = ['Des-I', 'Emp-I',
                   'DIA', 'MES', 'DIANOM', 'TIPOVUELO', 
                   'temporada_alta','periodo_dia', 'atraso_15']

In [ ]:
df_modelo = df[columnas_modelo]
df_modelo.head()

In [ ]:
df_modelo.atraso_15.value_counts(normalize=True)

In [ ]:
df_modelo.to_csv('df_modelo.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
df.head(3)

In [ ]:
df_conteo = df_raw.groupby(['Vlo-I'], as_index=False).\
           agg(unique_col=('Des-I', lambda x: x.nunique()),
               conca=('Des-I', lambda x: ','.join(x.unique()))).\
           sort_values(by=['unique_col'], ascending=[True])
df_conteo

In [ ]:
df_conteo = df_raw.groupby(['Vlo-I', 'Des-I' ], as_index=False).\
           agg(unique_col=('Des-I', 'count')).\
           sort_values(by=['Vlo-I'], ascending=[True])
df_conteo

In [ ]:
counts =  df_conteo['Vlo-I'].value_counts()

# Filtrar los resultados para mostrar sólo las categorías que aparecen más de una vez
duplicates = counts[counts > 1]

# Obtener una lista con las categorías duplicadas
duplicates_list = list(duplicates.index)
duplicates_list

In [ ]:
df_conteo[df_conteo['Vlo-I'].isin(duplicates_list)].head(50)

In [ ]:
df_conteo[df_conteo.unique_col>1]

In [ ]:
df_raw.groupby(['Ori-I'], as_index=False).\
           agg(unique_col=('SIGLAORI', lambda x: x.nunique()),
               conca=('SIGLAORI', lambda x: ','.join(x.unique()))).\
           sort_values(by=['Ori-I'], ascending=[True])

In [ ]:
df_raw.columns

In [ ]:
df_raw.groupby(['Emp-I'], as_index=False).\
           agg(unique_col=('OPERA', lambda x: x.nunique()),
               conca=('OPERA', lambda x: ','.join(x.unique()))).\
           sort_values(by=['Emp-I'], ascending=[True])

In [ ]:
df_conteo = df_raw.groupby(['Des-I'], as_index=False).\
           agg(unique_col=('SIGLADES', lambda x: x.nunique()),
               conca=('SIGLADES', lambda x: ','.join(x.unique()))).\
           sort_values(by=['Des-I'], ascending=[True])
df_conteo

In [ ]:
df_conteo[df_conteo.unique_col>1]

In [ ]:
df_conteo = df_raw.groupby(['Des-I', 'SIGLADES' ], as_index=False).\
           agg(unique_col=('SIGLADES', 'count')).\
           sort_values(by=['Des-I'], ascending=[True])
df_conteo

In [ ]:
counts =  df_conteo['Des-I'].value_counts()

# Filtrar los resultados para mostrar sólo las categorías que aparecen más de una vez
duplicates = counts[counts > 1]

# Obtener una lista con las categorías duplicadas
duplicates_list = list(duplicates.index)
duplicates_list

In [ ]:
df_conteo[df_conteo['Des-I'].isin(duplicates_list)]

In [ ]:
duplicates = df_conteo['Des-I' ].value_counts()>1).index()

In [ ]:
22508/(45698+22508)

- Periodo del dia: 3 (mañana, tarde y noche)
- Temporada alta: 2 
- Dia de la semana: 7
- Mes: 12
- Codigo Aerolinea: 